# Continuous Control

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!

### 1. Start the Environment

Run the next code cell to install a few packages.  This line will take a few minutes to run!

In [1]:
!pip -q install ./python

The environments corresponding to both versions of the environment are already saved in the Workspace and can be accessed at the file paths provided below.  

Please select one of the two options below for loading the environment.

In [2]:
from unityagents import UnityEnvironment
import numpy as np

# select this option to load version 1 (with a single agent) of the environment
#env = UnityEnvironment(file_name='/data/Reacher_One_Linux_NoVis/Reacher_One_Linux_NoVis.x86_64')

# select this option to load version 2 (with 20 agents) of the environment
env = UnityEnvironment(file_name='/data/Reacher_Linux_NoVis/Reacher.x86_64')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_size -> 5.0
		goal_speed -> 1.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


In [3]:
import torch
from collections import deque
from ddpg_multiple_agents import Agents

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [4]:
brain_name = env.brain_names[0]
brain = env.brains[brain_name]
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

agents = Agents(state_size=state_size, action_size=action_size, num_agents=num_agents, random_seed=0)


Number of agents: 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like: [  0.00000000e+00  -4.00000000e+00   0.00000000e+00   1.00000000e+00
  -0.00000000e+00  -0.00000000e+00  -4.37113883e-08   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00  -1.00000000e+01   0.00000000e+00
   1.00000000e+00  -0.00000000e+00  -0.00000000e+00  -4.37113883e-08
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   5.75471878e+00  -1.00000000e+00
   5.55726624e+00   0.00000000e+00   1.00000000e+00   0.00000000e+00
  -1.68164849e-01]


In [5]:
n_episodes = 500
max_t = 1000
print_every = 100

scores_deque = deque(maxlen=print_every)
scores = []
last_mean_score = 30

In [6]:
for i_episode in range(1, n_episodes+1):
    env_info = env.reset(train_mode=True)[brain_name]
    states = env_info.vector_observations
    
    agents.reset() # set the noise to zero
    score = np.zeros(num_agents)
    for t in range(max_t):
        
        
        actions = agents.act(states) 
        '''
        use local actor network to:
            1. predict action vector for each agent
            2. add OU noise to the actions of all agents
            3. clip actions to between -1 and +1 
        '''
        env_info = env.step(actions)[brain_name]
        next_states = env_info.vector_observations             # get the next states
        rewards = env_info.rewards                             # get the rewards
        dones = env_info.local_done                            # see if the episode has finished for any agent

        agents.step(states, actions, rewards, next_states, dones)
        '''
        1) add the experience tuples from all agents to a common replay buffer
        2) sample a batch of tuples for learning (if the replay buffer is large enough)
        -- a) Update the critic:
                i)   Get the next action by feeding next state into target actor 
                ii)  Get the action-value of next state/next action pair by feeding these into target critic - call this Q_next
                iii) Compute the 'actual' Q value for the current state/action pair as Q_current = r + γ*Q_next
                iv)  Get the predicted Q by feeding the current state and current action into the local critic - call this Q_pred
                v)   Compute the MSE loss between Q_current and Q_predicted, and update the weights of the local critic
        -- b) Update the actor:
                i)   Get the predicted actions for the current states based on the local actor
                ii)  Get the Q-values (expected reward) of these by passing the current states and predicted actions through the local critic
                iii) Use the negative mean of the Q-values as a loss to update the weights of the local actor
        -- c) Update the target networks
                Update the actor and critic target network weights by moving these slightly toward the local weights (soft-update)   
        '''
        states = next_states
        score += rewards
        if np.any(dones):
            break 
    scores_deque.append(np.mean(score)) # contains the last 'print_every' (=100) average scores for each agent
    scores.append(np.mean(score))
    print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)), end="")
    if i_episode % print_every == 0:
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))

    if len(scores_deque) == print_every:
        mean_score = np.mean(scores_deque)
        if mean_score > last_mean_score:
            torch.save(agents.actor_local.state_dict(), 'checkpoint_actor.pth')
            torch.save(agents.critic_local.state_dict(), 'checkpoint_critic.pth')
            last_mean_score = mean_score



Episode 100	Average Score: 30.91
Episode 200	Average Score: 37.04
Episode 300	Average Score: 36.71
Episode 400	Average Score: 36.35
Episode 500	Average Score: 36.43
